In [9]:
import pandas as pd
import numpy as np

In [10]:
df = pd.read_csv('/Users/JUSC/Documents/xgbsurv/xgbsurv/tests/simulation_data/survival_simulation_1000.csv')
risks = pd.read_csv('/Users/JUSC/Documents/xgbsurv/xgbsurv/tests/simulation_data/survival_simulation_preds1000.csv')

In [11]:
def breslow_estimator(log_hazard, time, event):
    #time, event = transform_back(y)
    risk_score = np.exp(log_hazard)

    is_sorted = lambda a: np.all(a[:-1] <= a[1:])

    if is_sorted(time) == False:
        order = np.argsort(time, kind="mergesort")
        time = time[order]
        event = event[order]
        risk_score = risk_score[order]

    uniq_times = np.unique(time)
    idx = np.digitize(time, np.unique(time))
    breaks = np.flatnonzero(np.concatenate(([1], np.diff(idx))))
    # numpy diff nth discrete difference over index, add 1 at the beginning
    # flatnonzero return indices that are nonzero in flattened version
    n_events = np.add.reduceat(event, breaks, axis=0)

    # consider removing zero rows, would this be the right approach?
    risk_matrix = np.unique((np.outer(time,time)>=np.square(time)).astype(int).T, axis=0)
    denominator = np.sum(risk_score[None,:]*risk_matrix,axis=1)[::-1]     

    cum_hazard_baseline = np.cumsum(n_events / denominator)
    baseline_survival = np.exp(-cum_hazard_baseline)
    return uniq_times, cum_hazard_baseline, baseline_survival

In [54]:
def breslow_estimator_loop(    predictor: np.array,
    time: np.array,
    event: np.array

):
    exp_predictor: np.array = np.exp(predictor)
    local_risk_set: float = np.sum(exp_predictor)
    event_mask: np.array = event.astype(np.bool_)
    n_unique_events: int = np.unique(time[event_mask]).shape[0]
    cumulative_baseline_hazards: np.array = np.zeros(n_unique_events)
    n_events_counted: int = 0
    local_death_set: int = 0
    accumulated_risk_set: float = 0
    previous_time: float = time[0]

    for _ in range(len(time)):
        sample_time: float = time[_]
        sample_event: int = event[_]
        sample_predictor: float = exp_predictor[_]

        if sample_time > previous_time and local_death_set:
            cumulative_baseline_hazards[n_events_counted] = local_death_set / (
                local_risk_set
            )

            local_death_set = 0
            local_risk_set -= accumulated_risk_set
            accumulated_risk_set = 0
            n_events_counted += 1

        if sample_event:
            local_death_set += 1
        accumulated_risk_set += sample_predictor
        previous_time = sample_time

    cumulative_baseline_hazards[n_events_counted] = local_death_set / (
        local_risk_set
    )

    return (
        np.unique(time[event_mask]),
        np.cumsum(cumulative_baseline_hazards),
    )

In [55]:
log_hazard, time, event = risks.to_numpy(), df.time.to_numpy(), df.event.to_numpy()

In [56]:
res_loop = breslow_estimator_loop( log_hazard, time, event)
res = breslow_estimator(log_hazard, time, event)


In [57]:
# times comparison
times_loop = res_loop[0]
print(times_loop.shape)
times = res[0]
print(times.shape)

(370,)
(1000,)


In [58]:
# hazards comparison
cum_hazard_loop = res_loop[1]
print(cum_hazard_loop.shape)
cum_hazard = res[1]
print(cum_hazard.shape)

(370,)
(1000,)
